In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from speedy_utils.all import *

setup_logger("D")

DEBUG    | utils_print.py:265 (setup_logger) - Logging set to DEBUG


In [33]:





def slow_func(x):
    # try:
    time.sleep(1)  # sleeps for `x` seconds
    logger.info(f"{x=}")
    return x
    # except KeyboardInterrupt:
    #     logger.error("KB Interrupted")
    # except Exception as e:
    #     logger.error(f"Interrupted: {e}")
        

# write a decorator that raises an exception if the function is interrupted by the user


slow_func(1)

INFO     | 3873382941.py:4 (slow_func) - x=1


1

In [ ]:
setup_logger("i")

results = None
results = multi_thread_in_sub_process(
    func=slow_func,
    orig_inputs=range(20),
    workers=10,
    desc="Demo",
    verbose=False
)

Demo:   0%|                                                      | 0/20 [00:00<?, ?it/s]

INFO     | 3873382941.py:4 (slow_func) - x=1
INFO     | 3873382941.py:4 (slow_func) - x=2
INFO     | 3873382941.py:4 (slow_func) - x=4


Demo:   0%|                                           | 0/20 [00:01<?, ?it/s, SUCCESS=2]

INFO     | 3873382941.py:4 (slow_func) - x=8


Demo:  10%|███▌                               | 2/20 [00:01<00:09,  1.96it/s, SUCCESS=2]

INFO     | 3873382941.py:4 (slow_func) - x=7
INFO     | 3873382941.py:4 (slow_func) - x=5
INFO     | 3873382941.py:4 (slow_func) - x=3
INFO     | 3873382941.py:4 (slow_func) - x=0
INFO     | 3873382941.py:4 (slow_func) - x=9
INFO     | 3873382941.py:4 (slow_func) - x=6


ERROR    | thread.py:302 (multi_thread) - [MAIN PROCESS] Keyboard interrupt detected stop `slow_func` execution.


In [ ]:
import time
import logging
from tqdm import tqdm
from threading import Semaphore
from speedy_utils.all import *
from fastcore.all import *

# Configure logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def f(x):
    time.sleep(1)  # Simulate work
    logger.info(f"{x=}")
    return x

def multi_thread_in_sub_process(func, orig_inputs, workers=4, desc=None, verbose=False):
    results = []
    desc = desc or "Processing"
    
    semaphore = Semaphore(workers)  # Limit concurrent threads

    @threaded
    def _f(i):
        with semaphore:  # Ensure only 'workers' threads run at a time
            return func(i)

    threads = [_f(i) for i in orig_inputs]  # Start threads

    for t in tqdm(threads, desc=desc, total=len(orig_inputs)):  
        t.join()  # Wait for thread completion
        results.append(t.result)  # Collect result

    return results

def multi_thread(func, orig_inputs, workers=4, desc=None, verbose=False):
    _mt = threaded(process=True)(multi_thread_in_sub_process)
    return _mt(func, orig_inputs, workers, desc, verbose)
# Example usage
inputs = list(range(10))  # Example inputs
results = multi_thread_in_sub_process(f, inputs, workers=3, desc="Running tasks")

In [18]:
thread = f(1)
thread.join()
thread.result

2025-02-17 07:03:26.034 | INFO     | __main__:f:4 - x=1


1